In [1]:
import pandas as pd
from itertools import product
from tqdm.notebook import tqdm

import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.storage import (
    get_demand_model_data,
    get_demand_orig_dest_model_data,
)
from modules.config import *
from modules.svm import *

In [2]:
all_possible_metas = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100],  'gamma': [-1],                          'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['rbf'],    'C': [0.1, 1, 10, 100],  'gamma': [0.1, 0.01, 0.001, 0.0001],    'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['poly'],   'C': [1, 10, 100, 1000], 'gamma': [-1],                          'degree': [2, 3, 4, 5], 'max_iter': [1000000]}
]

In [3]:
execute_stage(
    'first_stage',
    SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
    SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
    all_possible_metas,
    get_demand_model_data,
)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
results = pd.read_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False).head(2)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_max_iter,params,...,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,param_gamma,param_degree
28,2,9999,78.376207,0.598169,14.459992,1.564005,100.0,rbf,1000000,"{'C': 100.0, 'degree': None, 'gamma': 0.01, 'k...",...,-5.337175,-5.567812,-5.539053,-5.645006,0.591507,0,8,6,0.01,NaN
52,2,9999,74.175769,0.720615,8.921712,0.536912,100.0,poly,1000000,"{'C': 100.0, 'degree': 3.0, 'gamma': None, 'ke...",...,-14.650536,-11.141603,-13.478221,-12.700328,1.453161,0,8,6,NaN,3.0


In [ ]:
results.sort_values(by='mean_test_score', ascending=False).groupby('param_kernel')[[
    'mean_test_score',
    'param_C',
    'param_degree',
    'param_gamma',
]].first()

,mean_test_score,param_C,param_degree,param_gamma
param_kernel,,,,
linear,-45.875257,0.1,NaN,NaN
poly,-33.056210,100.0,3.0,NaN
rbf,-28.179017,100.0,NaN,0.01


In [ ]:
results.groupby('param_kernel').mean_fit_time.sum() / 60

param_kernel
linear    5.949147
poly      6.084356
rbf       4.065095
Name: mean_fit_time, dtype: float64

In [ ]:
resolutions = list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS))
resolutions = resolutions + (ADDITIONAL_PREDICTIVE_RESOLUTIONS)
resolutions

[(7, 1), (7, 2), (7, 6), (7, 24), (8, 1), (8, 2), (8, 6), (8, 24), (9, 24)]

In [ ]:
for h3_res, time_interval_length in tqdm(resolutions):
    execute_stage(
        'second_stage',
        SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
        SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
        h3_res,
        time_interval_length,
        all_possible_metas,
        get_demand_model_data,
    )

  0%|          | 0/9 [00:00<?, ?it/s]

c:\Dev\miniconda\envs\AAA_MAGMA\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Dev\miniconda\envs\AAA_MAGMA\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [ ]:
results = pd.read_parquet(SVM_SECOND_STAGE_DEMAND_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,param_kernel,param_max_iter,...,std_train_score,n_iter,h3_res,time_interval_length,test_mse,test_rmse,test_mae,test_non_zero_mape,test_zero_accuracy,param_gamma
4,0,10000,14.925890,0.998992,2.864986,0.310409,1000.0,3.0,poly,1000000,...,0.000040,21890,8,1,0.744513,0.862852,0.301780,0.689565,0.893975,NaN
0,0,10000,20.947026,1.448813,3.189637,1.373549,1000.0,3.0,poly,1000000,...,0.004270,69597,7,1,3.872556,1.967881,0.799453,0.499394,0.733520,NaN
5,0,10000,27.908084,1.115601,4.726018,1.125819,1000.0,3.0,poly,1000000,...,0.001635,74212,8,2,5.334046,2.309555,0.849459,0.651641,0.806883,NaN
8,0,10000,90.931578,3.435078,17.598868,2.556205,1000.0,3.0,poly,1000000,...,0.369777,84715,9,24,16.568389,4.070429,1.826414,0.561964,0.819742,NaN
1,0,10000,31.952004,0.888609,3.003483,0.747905,1000.0,3.0,poly,1000000,...,0.043166,280964,7,2,32.923326,5.737885,2.663975,0.452378,0.367151,NaN
6,0,10000,46.653558,2.103604,8.595032,3.749019,1000.0,3.0,poly,1000000,...,0.540698,186630,8,6,26.493718,5.147205,2.071947,0.497319,0.596576,NaN
7,0,10000,49.889175,2.427127,6.717005,1.836598,1000.0,3.0,poly,1000000,...,1.197659,228141,8,24,56.152276,7.493482,3.038408,0.305815,0.684384,NaN
2,0,10000,103.081549,6.155593,1.816785,0.738815,1000.0,3.0,poly,1000000,...,2.229625,1000000,7,6,156.158906,12.496356,5.539407,0.293954,0.351401,NaN
3,0,8797,92.581563,5.002434,1.262368,0.382321,1000.0,3.0,poly,1000000,...,9.965680,1000000,7,24,391.729112,19.792148,7.705526,0.181783,0.566606,NaN


In [ ]:
results_first_stage = pd.read_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH)
results_second_stage = pd.read_parquet(SVM_SECOND_STAGE_DEMAND_RESULTS_PATH)
results_first_stage.to_parquet(os.path.join(REPO_DATA_DIR_PATH, f"{str(SVM_MAX_TRAIN_SET_SIZE)}_svm_first_stage_demand_results.parquet"))
results_second_stage.to_parquet(os.path.join(REPO_DATA_DIR_PATH, f"{str(SVM_MAX_TRAIN_SET_SIZE)}_svm_second_stage_demand_results.parquet"))

In [ ]:
execute_stage(
    'second_stage',
    SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
    SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
    int(f"{h3_res}{h3_res}"),
    time_interval_length,
    all_possible_metas,
    lambda _, __, : get_demand_orig_dest_model_data(),
)